# EGM

**Table of contents**<a id='toc0_'></a>    
- 1. [Imports](#toc1_)    
- 2. [Model](#toc2_)    
- 3. [Numba](#toc3_)    
  - 3.1. [Compile](#toc3_1_)    
  - 3.2. [Euler-error](#toc3_2_)    
  - 3.3. [Transfer func](#toc3_3_)    
  - 3.4. [Multiple Rs and transfer funcs](#toc3_4_)    
  - 3.5. [Save](#toc3_5_)    
  - 3.6. [Time](#toc3_6_)    
- 4. [C++](#toc4_)    
  - 4.1. [Compile](#toc4_1_)    
  - 4.2. [Time](#toc4_2_)    
  - 4.3. [No threading](#toc4_3_)    
  - 4.4. [R](#toc4_4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Imports](#toc0_)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import time
import matplotlib.pyplot as plt
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

In [3]:
from BufferStockModel import BufferStockModelClass
from BufferStockModelEGM import BufferStockModelEGMClass

## 2. <a id='toc2_'></a>[Model](#toc0_)

In [4]:
model = BufferStockModelEGMClass(name='test',par={'Nstates_fixed':0})

**Check same random numbers are used in DP and DL:**

In [5]:
model_DL = BufferStockModelClass(algoname='DeepSimulate',device='cpu',par={'Nstates_fixed':0,'full':True})
assert np.allclose(model.sim.states[0],model_DL.sim.states[0].numpy())
assert np.allclose(model.sim.shocks,model_DL.sim.shocks.numpy())

## 3. <a id='toc3_'></a>[Numba](#toc0_)

### 3.1. <a id='toc3_1_'></a>[Compile](#toc0_)

In [ ]:
t0 = time.perf_counter()
model.solve_EGM()
print(f'Compile: {time.perf_counter()-t0:.1f} seconds')

In [7]:
model.simulate_R()

In [ ]:
print(f'R = {model.sim.R:12.8f}')

### 3.2. <a id='toc3_2_'></a>[Euler-error](#toc0_)

In [9]:
model.compute_euler_errors()

In [ ]:
from BufferStockModelEGM import mean_log10_euler_error_working_EGM
print(f'Mean log10 Euler error: {mean_log10_euler_error_working_EGM(model):.1f}')

### 3.3. <a id='toc3_3_'></a>[Transfer func](#toc0_)

In [11]:
model.compute_transfer_func()

In [ ]:
fig = plt.figure
ax = plt.subplot(1,1,1)
ax.plot(model.egm.transfer_grid,model.sim.R_transfer,'-o');

### 3.4. <a id='toc3_4_'></a>[Multiple Rs and transfer funcs](#toc0_)

In [13]:
if model.sim.reps > 0: model.simulate_Rs()

In [14]:
if model.sim.reps > 0: 
    fig = plt.figure
    ax = plt.subplot(1,1,1)
    for rep in range(model.sim.reps):
        ax.plot(model.egm.transfer_grid,model.sim.R_transfers[rep],'-o');

### 3.5. <a id='toc3_5_'></a>[Save](#toc0_)

In [15]:
model.save('test.pkl')

In [16]:
os.remove('test.pkl')

### 3.6. <a id='toc3_6_'></a>[Time](#toc0_)

In [ ]:
for _ in range(3):
    
    t0 = time.perf_counter()
    model.solve_EGM()
    print(f'Solve: {time.perf_counter()-t0:.2f} seconds')

## 4. <a id='toc4_'></a>[C++](#toc0_)

### 4.1. <a id='toc4_1_'></a>[Compile](#toc0_)

In [ ]:
t0 = time.perf_counter()
model.link_to_cpp(do_print=False)
print(f'Compile: {time.perf_counter()-t0:.1f} seconds')

### 4.2. <a id='toc4_2_'></a>[Time](#toc0_)

In [ ]:
print(f'cppthreads = {model.par.cppthreads}')
for _ in range(3):

    t0 = time.perf_counter()
    model.cpp.solve_all(model.par,model.egm)
    EGM_time = time.perf_counter()-t0
    print(f'Solve: {EGM_time:.2f} seconds')

### 4.3. <a id='toc4_3_'></a>[No threading](#toc0_)

In [ ]:
for cppthreads in [1]:

    model.par.cppthreads = cppthreads 
    print(f'cppthreads = {model.par.cppthreads}')

    for _ in range(3):

        t0 = time.perf_counter()
        model.cpp.solve_all(model.par,model.egm)
        EGM_time = time.perf_counter()-t0
        print(f'Solve: {EGM_time:.2f} seconds')
    
    print('')


### 4.4. <a id='toc4_4_'></a>[R](#toc0_)

In [ ]:
model.simulate_R() # same states and shocks as in .sim
print(f'R = {model.sim.R:12.8f}')